In [0]:
import json
from pyspark.sql import functions as F

In [0]:
def get_job_1900_bronze():
    bronze = (
        spark.read.option("multiline", "true")
        .format("json")
        .load("/Volumes/dev/job_prospects/job_1900_bronze")
    )
    return bronze

bronze = get_job_1900_bronze()

In [0]:
def parse_number(col):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), None)
        .when(
            F.col(col).rlike(r"^\s*\d+(\.\d+)?\s*[kK]"),
            (F.regexp_extract(col, r"(\d+(\.\d+)?)", 1).cast("double") * 1000).cast(
                "int"
            ),
        )
        .when(F.col(col).rlike(r"\d+"), F.regexp_extract(col, r"(\d+)", 1).cast("int"))
        .otherwise(None)
    )


def parse_min_salary(col):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), None)
        .when(F.lower(F.trim(F.col(col))) == "thoả thuận", None)
        .when(F.col(col).rlike(r"(?i)^t[ơo]i\s*|^tới\s*"), None)
        .when(
            F.col(col).rlike(r"(?i)trên\s*([0-9]+(?:[.,][0-9]+)?)\s*([^\d\s-]+)"),
            F.concat(
                F.regexp_extract(col, r"(?i)trên\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                F.lit(" "),
                F.regexp_extract(
                    col, r"(?i)trên\s*[0-9]+(?:[.,][0-9]+)?\s*([^\d\s-]+)", 1
                ),
            ),
        )
        .when(
            F.col(col).rlike(
                r"^\s*([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)\s*([^\d\s-]+)"
            ),
            F.concat(
                F.regexp_extract(col, r"^\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                F.lit(" "),
                F.regexp_extract(
                    col,
                    r"^\s*[0-9]+(?:[.,][0-9]+)?\s*-\s*[0-9]+(?:[.,][0-9]+)?\s*([^\d\s-]+)",
                    1,
                ),
            ),
        )
        .otherwise(
            F.trim(
                F.concat(
                    F.regexp_extract(col, r"^\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                    F.lit(" "),
                    F.regexp_extract(
                        col, r"^\s*[0-9]+(?:[.,][0-9]+)?\s*([^\d\s-]+)", 1
                    ),
                )
            )
        )
    )


def parse_max_salary(col):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), None)
        .when(F.lower(F.trim(F.col(col))) == "thoả thuận", None)
        .when(F.col(col).rlike(r"(?i)^trên"), None)
        .when(
            F.col(col).rlike(r"(?i)^t[ơo]i\s*([0-9]+(?:[.,][0-9]+)?)\s*(triệu|usd)"),
            F.concat(
                F.regexp_extract(col, r"(?i)^t[ơo]i\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                F.lit(" "),
                F.regexp_extract(
                    col, r"(?i)^t[ơo]i\s*[0-9]+(?:[.,][0-9]+)?\s*(triệu|usd)", 1
                ),
            ),
        )
        .when(
            F.col(col).rlike(
                r"^\s*([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)\s*([^\d\s-]+)"
            ),
            F.concat(
                F.regexp_extract(
                    col, r"^\s*([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)", 2
                ),
                F.lit(" "),
                F.regexp_extract(
                    col,
                    r"^\s*[0-9]+(?:[.,][0-9]+)?\s*-\s*([0-9]+(?:[.,][0-9]+)?)\s*([^\d\s-]+)",
                    2,
                ),
            ),
        )
        .otherwise(
            F.trim(
                F.concat(
                    F.regexp_extract(col, r"([0-9]+(?:[.,][0-9]+)?)", 1),
                    F.lit(" "),
                    F.regexp_extract(col, r"([0-9]+(?:[.,][0-9]+)?)\s*([^\d\s-]+)", 2),
                )
            )
        )
    )


def convert_to_usd(col):
    return (
        F.when(
            F.col(col).rlike(r"(?i)([0-9]+(?:[.,][0-9]+)?)\s*triệu"),
            (
                (
                    F.regexp_replace(
                        F.regexp_extract(F.col(col), r"([0-9]+(?:[.,][0-9]+)?)", 1),
                        ",",
                        ".",
                    ).cast("double")
                    * 1_000_000
                )
                * 0.000038
            ).cast("int"),
        )
        .when(
            F.col(col).rlike(r"(?i)([0-9]+(?:[.,][0-9]+)?)\s*usd"),
            F.regexp_replace(
                F.regexp_extract(F.col(col), r"([0-9]+(?:[.,][0-9]+)?)", 1), ",", "."
            )
            .cast("double")
            .cast("int"),
        )
        .otherwise(None)
    )


def parse_float(col):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), None)
        .when(
            F.col(col).rlike(r"^\s*-?\d+([.,]\d+)?\s*$"),
            F.regexp_replace(F.col(col), ",", ".").cast("double"),
        )
        .otherwise(None)
    )


def parse_min_experience(col):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), None)
        .when(F.lower(F.trim(F.col(col))) == "không yêu cầu", F.lit(0))
        .when(F.col(col).rlike(r"(?i)^t[ơo]i\s*[0-9]+(?:[.,][0-9]+)?\s*năm"), F.lit(0))
        .when(F.col(col).rlike(r"(?i)^t[ơo]i\s*năm"), F.lit(0))
        .when(F.col(col).rlike(r"(?i)^tới\s*[0-9]+(?:[.,][0-9]+)?\s*năm"), F.lit(0))
        .when(
            F.col(col).rlike(r"(?i)^trên\s*([0-9]+(?:[.,][0-9]+)?)\s*năm"),
            F.regexp_replace(
                F.regexp_extract(col, r"(?i)^trên\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                ",",
                ".",
            ).cast("int"),
        )
        .when(
            F.col(col).rlike(
                r"([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)\s*năm"
            ),
            F.regexp_replace(
                F.regexp_extract(
                    col, r"([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)", 1
                ),
                ",",
                ".",
            ).cast("int"),
        )
        .when(
            F.col(col).rlike(r"([0-9]+(?:[.,][0-9]+)?)\s*năm"),
            F.regexp_replace(
                F.regexp_extract(col, r"([0-9]+(?:[.,][0-9]+)?)\s*năm", 1), ",", "."
            ).cast("int"),
        )
        .otherwise(None)
    ).cast("int")


def parse_max_experience(col):
    return (
        F.when(F.col(col).isNull() | (F.trim(F.col(col)) == ""), None)
        .when(F.lower(F.trim(F.col(col))) == "không yêu cầu", None)
        .when(
            F.col(col).rlike(r"(?i)^t[ơo]i\s*[0-9]+(?:[.,][0-9]+)?\s*năm"),
            F.regexp_replace(
                F.regexp_extract(col, r"(?i)^t[ơo]i\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                ",",
                ".",
            ).cast("int"),
        )
        .when(F.col(col).rlike(r"(?i)^t[ơo]i\s*năm"), None)
        .when(
            F.col(col).rlike(r"(?i)^tới\s*[0-9]+(?:[.,][0-9]+)?\s*năm"),
            F.regexp_replace(
                F.regexp_extract(col, r"(?i)^tới\s*([0-9]+(?:[.,][0-9]+)?)", 1),
                ",",
                ".",
            ).cast("int"),
        )
        .when(F.col(col).rlike(r"(?i)^trên\s*([0-9]+(?:[.,][0-9]+)?)\s*năm"), None)
        .when(
            F.col(col).rlike(
                r"([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)\s*năm"
            ),
            F.regexp_replace(
                F.regexp_extract(
                    col, r"([0-9]+(?:[.,][0-9]+)?)\s*-\s*([0-9]+(?:[.,][0-9]+)?)", 2
                ),
                ",",
                ".",
            ).cast("int"),
        )
        .when(
            F.col(col).rlike(r"([0-9]+(?:[.,][0-9]+)?)\s*năm"),
            F.regexp_replace(
                F.regexp_extract(col, r"([0-9]+(?:[.,][0-9]+)?)\s*năm", 1), ",", "."
            ).cast("int"),
        )
        .otherwise(None)
    ).cast("int")


def parse_area(col):
    cleaned = F.regexp_replace(F.col(col), r"Địa điểm làm việc:\s*", "")
    cleaned = F.regexp_replace(cleaned, r"\s*-\s*Việc làm tại\s*", "|")
    cleaned = F.regexp_replace(cleaned, r"(?i)Việc làm tại\s*", "")
    cleaned = F.regexp_replace(cleaned, r"\s*-\s*", "|")
    cleaned = F.regexp_replace(cleaned, r"375 Đường Ngọc Hồi|Thị Trấn Văn Điển|Huyện Thanh Trì|TP Hà Nội", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"(\[\")|(\"\])|(\")", "")
    cleaned = F.regexp_replace(cleaned, r"^\s*\d+[^\,]*,\s*", "")
    cleaned = F.regexp_replace(cleaned, r":", "")
    for i in range(1, 18):
        cleaned = F.regexp_replace(
            cleaned,
            r"(?i)^\s*((\d+)|(địa chỉ)|(ấp)|(huyện)|(quận)|(xã)|(phường)|(p\.)|(q\.)|(khóm)|(thị tr)|(tại hộ)|(phố)|(đài)|(uỷ)|(Ủy)|(UBND)|(Hđnd)|(tại\str)|(km)|(bệnh viện)|(hội trường)|(dự kiến)|(văn phòng)|(bộ phận)|(cảng)|(trường)|(nhà)|(quốc lộ)|(cơ sở)|(trụ sở)|(học v)|(\(tầng)|(thôn)|(P\sH)|(thị xã)|(chung cư)|(ố \d)|(tổ)|(toà)|(Tòa)|(nộp)|(tại phòng)|(tại bệnh)|(tại tầ)|(trung tâm)|(công ty)|(chi nhánh)|(khoa y)|(Đhqg)|(P2809)|(tt)|(ban bồi)|(lầu)|(khối)|(số)|(ban\stổ)|(khoa\sdu)|(Số)|(Số)|(khu)|(ngõ)|(tiểu khu)|(\+)|(kp)|(đường)|(phòng)|(trường)|(tân hưng,\sq7)|(đại học)|(thành phố Thủ Đức)|(GÒ VẤP)|(chi cục)|(thị xã)|(nhận phiếu)|(trong giờ)|(đại học)|(khoa tài)|(trường)|(điện thoại)|(địa chỉ)|(sở y tế)|(sở văn hoá)|(tại văn)|(khoa lu)|(tạp chí)|(Tầng)|(tháp)|(xóm)|(Các ứng))[^,]*,\s*",
            "",
        )
    cleaned = F.regexp_replace(cleaned, r"thành phố Hà Nội\.", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"(?i)tỉnh Kon Tum*tỉnh Kon Tum", "Kon Tum")
    cleaned = F.regexp_replace(cleaned, r"(?i)(TP. Hồ Chí Minh)|(TPHCM)|(TP.HCM)|(TP. HCM)|(TP\.Hồ Chí Minh)|(HCM)|(Hồ Chí MinhC)", "Hồ Chí Minh")
    cleaned = F.regexp_replace(cleaned, r"(?i)tỉnh\s", "")
    cleaned = F.regexp_replace(cleaned, r"(?i)thành phố\s([^,]+), \1", r"\1")
    cleaned = F.regexp_replace(cleaned, r"(?i)\s*(Đắk Nông).*\1", r"\1")
    cleaned = F.regexp_replace(cleaned, r"Trường Đại học Việt Nhật .* Hà Nội \(ĐHQGHN\)", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"\s*\([^\)]*\)", "")
    cleaned = F.regexp_replace(cleaned, r"Dự kiến tại Trường THPT số 2 Lào Cai", "Lào Cai")
    cleaned = F.regexp_replace(cleaned, r"thành phố Hà Nội và cơ sở 2 tại xã Tân Minh, huyện Sóc Sơn, Hà Nội", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"Nhà số 27 – Đường 800A|Nghĩa Đô – Cầu giấy", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"thành phố Hà Nội hoặc gửi theo đường bưu chính theo địa chỉ trên.", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"    dự kiến tại trường THCS Trần Văn Ơn, số 03 Phạm Phú Thứ, phường Hạ Lý, quận Hồng Bàng, thành phố Hải Phòng.", "Hải Phòng")
    cleaned = F.regexp_replace(cleaned, r"Nộp trực tiếp tại Phòng Tổ chức – Hành chính, Trung tâm Kiểm soát bệnh tật Quảng Trị.", "Quảng Trị")
    cleaned = F.regexp_replace(cleaned, r"(?i)(TP. Hồ Chí Minh)|(TPHCM)|(TP.HCM)|(TP. HCM)|(TP\.Hồ Chí Minh)|(HCM)|(Hồ Chí MinhC)", "Hồ Chí Minh")
    cleaned = F.regexp_replace(cleaned, r"(?i)Tầng 9\|Khu trung tâm hành chính Số 36\|Trần Phú\|Phường 4\|TP Đà Lạt\.", "Đà Lạt")
    cleaned = F.regexp_replace(cleaned, r"(?i)^quận \d$", "Hồ Chí Minh")
    cleaned = F.regexp_replace(cleaned, r"(?i)(tp\.)|(TP\s)", "")
    cleaned = F.regexp_replace(cleaned, r"Hồ Chí Minh – Điện thoại 028 3965 0197\.", "Hồ Chí Minh")
    cleaned = F.regexp_replace(cleaned, r"Quận 5\|Hồ Chí Minh", "Hồ Chí Minh")
    cleaned = F.regexp_replace(cleaned, r"(?i)(tp\.)|(TP\s)|(thành\sphố\s)", "")
    cleaned = F.regexp_replace(cleaned, r"(Quảng Ninh hoặc gửi theo đường bưu chính\.)|(Đinh Tiên Hoàng, phường Quang Trung, Uông Bí, Quảng Ninh)", "Quảng Ninh")
    cleaned = F.regexp_replace(cleaned, r"Tân An, Long An; từ ngày 01/3/2025 nhận hồ sơ tại trụ sở mới – Tuyến tránh Quốc lộ 1A, phường 4, Tân An, Long An\.", "Long An")
    cleaned = F.regexp_replace(cleaned, r"(?i)TP[^,]*Bắc Kạn, Bắc Kạn.", "Bắc Kạn")
    cleaned = F.regexp_replace(cleaned, r"(?i)TP\r\n(Bắc Kạn), \1\)", r"\1")
    cleaned = F.regexp_replace(cleaned, r"(?i)Số 246\|lộ 943\|Phường Mỹ Hòa\| Long Xuyên\|An Giang", "An Giang")
    cleaned = F.regexp_replace(cleaned, r"(?i)\+ Số 48 Tô Hiệu\|Hà Đông\|Hà Nội.*Hà Nội", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"(?i)169 Nguyễn Ngọc Vũ\|Trung Hòa\|Cầu Giấy\|Hà Nội", "Hà Nội")
    cleaned = F.regexp_replace(cleaned, r"(\)\.)|(\.)", "")
    cleaned = F.regexp_replace(cleaned, r"(?i)viện\s([^,]+)", "")
    return F.trim(cleaned)



def parse_employment_type(col):
    return F.array_distinct(
        F.transform(F.split(F.col(col), r"\s*,\s*|\s*;\s*"), lambda x: F.trim(x))
    )


def parse_gender(col):
    return F.when(
        F.lower(F.trim(F.col(col))).isin("nam", "nữ"),
        F.initcap(F.lower(F.trim(F.col(col)))),
    ).otherwise("Không yêu cầu")


def parse_company_size(col):
    return F.when(F.col(col) == "__", None).otherwise(F.col(col))


df = bronze.withColumns(
    {
        "id": F.monotonically_increasing_id(),
        "timestamp": F.to_date("timestamp"),
        "number_of_reviews": parse_number("number_of_reviews"),
        "number_of_jobs": parse_number("number_of_jobs"),
        "views": parse_number("views"),
        "salary_low": parse_min_salary("salary"),
        "salary_high": parse_max_salary("salary"),
        "employment_type": parse_employment_type("employment_type"),
        "posted_at": F.to_date(F.col("posted_at"), "dd/MM/yyyy"),
        "quantity": parse_number("quantity"),
        "application_deadline": F.to_date(
            F.col("application_deadline"), "dd/MM/yyyy"
        ),
        "minimum_experience": parse_min_experience("experience"),
        "maximum_experience": parse_max_experience("experience"),
        "gender": parse_gender("gender"),
        # gender
        # "job_description": F.when(
        #     F.col("job_description").isNotNull(),
        #     F.expr("concat_ws('\n', job_description)")
        # ).otherwise(None),
        "company_size": parse_company_size("company_size"),
        "company_rating": parse_float("company_rating"),
        "area": parse_area("area"),
    }
)
df = df.withColumns(
    {
        "quantity": F.when(F.col("quantity") > 5, F.lit(5)).otherwise(
            F.col("quantity")
        ),
        "salary_low": convert_to_usd("salary_low"),
        "salary_high": convert_to_usd("salary_high"),
    }
)
# display(df.select(F.explode("area").alias("area")).distinct())
# display(df.select("area").distinct())

In [0]:
area_list = [
    "Cao Bằng",
    "Sơn La",
    "Lai Châu",
    "Lạng Sơn",
    "Tuyên Quang",
    "Lào Cai",
    "Thái Nguyên",
    "Điện Biên",
    "Phú Thọ",
    "Bắc Ninh",
    "Hà Nội",
    "Quảng Ninh",
    "Hải Phòng",
    "Hưng Yên",
    "Ninh Bình",
    "Thanh Hóa",
    "Nghệ An",
    "Hà Tĩnh",
    "Quảng Trị",
    "Huế",
    "Đà Nẵng",
    "Quảng Ngãi",
    "Gia Lai",
    "Đắk Lắk",
    "Khánh Hòa",
    "Lâm Đồng",
    "Đồng Nai",
    "Thành phố Hồ Chí Minh",
    "Tây Ninh",
    "Đồng Tháp",
    "Vĩnh Long",
    "Cần Thơ",
    "An Giang",
    "Cà Mau"
]

In [0]:
df.createOrReplaceTempView("df_view")
area_list_str = json.dumps(area_list)

classified_area_df = spark.sql(
    f"""
    SELECT *,
        from_json(
            ai_query(
                'databricks-meta-llama-3-3-70b-instruct',
                'Given the following area names as string separated by "," or "|", classify each into one or more of the following valid areas:\n {area_list_str}\nReturn a list of matched area(s) for each input in only an JSON array format, no comments whatsoever. Input:\n' || area || "For example, if the input is 'Hà Nội,quận 1, TP. HCM|Hải Phòng', the output should be a JSON format: ['Hà Nội', 'TP. HCM', 'Hải Phòng']"
            ),
            'ARRAY<STRING>'
        ) AS area_classified
    FROM df_view
    """
)
silver.withColumn(
    "area", classified_area_df["area_classified"]
).drop("area_classified")

In [0]:
silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("dev.job_prospects.job_1900_silver")

In [0]:
# Test cases for parse_number
test_data = [
    ("100", 100),
    ("2k", 2000),
    ("3.5K", 3500),
    (" 7 K ", 7000),
    ("12", 12),
    ("0", 0),
    ("", None),
    (None, None),
    ("abc", None),
    ("1.2k", 1200),
    ("999", 999),
    ("5.7k", 5700),
]

test_df = spark.createDataFrame(test_data, ["input", "expected"])
result_df = test_df.withColumn("parsed", parse_number("input"))

mismatches = result_df.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert mismatches.count() == 0, "parse_number failed for some test cases"

# Test cases for parse_min_salary
min_salary_data = [
    ("10 triệu", "10 triệu"),
    ("15 usd", "15 usd"),
    ("Trên 20 triệu", "20 triệu"),
    ("5-10 triệu", "5 triệu"),
    ("", None),
    (None, None),
    ("thoả thuận", None),
    ("Tới 30 triệu", None),
]
min_salary_df = spark.createDataFrame(min_salary_data, ["input", "expected"])
min_salary_result = min_salary_df.withColumn("parsed", parse_min_salary("input"))
min_salary_mismatches = min_salary_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert min_salary_mismatches.count() == 0, "parse_min_salary failed for some test cases"

# Test cases for parse_max_salary
max_salary_data = [
    ("10 triệu", "10 triệu"),
    ("15 usd", "15 usd"),
    ("Tới 20 triệu", "20 triệu"),
    ("5-10 triệu", "10 triệu"),
    ("", None),
    (None, None),
    ("thoả thuận", None),
    ("Trên 30 triệu", None),
]
max_salary_df = spark.createDataFrame(max_salary_data, ["input", "expected"])
max_salary_result = max_salary_df.withColumn("parsed", parse_max_salary("input"))
max_salary_mismatches = max_salary_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert max_salary_mismatches.count() == 0, "parse_max_salary failed for some test cases"

# Test cases for convert_to_usd
usd_data = [
    ("10 triệu", int(10_000_000 * 0.000038)),
    ("15 usd", 15),
    ("", None),
    (None, None),
    ("abc", None),
]
usd_df = spark.createDataFrame(usd_data, ["input", "expected"])
usd_result = usd_df.withColumn("parsed", convert_to_usd("input"))
usd_mismatches = usd_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert usd_mismatches.count() == 0, "convert_to_usd failed for some test cases"

# Test cases for parse_float
float_data = [
    ("4.5", 4.5),
    ("3,2", 3.2),
    ("", None),
    (None, None),
    ("abc", None),
]
float_df = spark.createDataFrame(float_data, ["input", "expected"])
float_result = float_df.withColumn("parsed", parse_float("input"))
float_mismatches = float_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert float_mismatches.count() == 0, "parse_float failed for some test cases"

# Test cases for parse_min_experience
min_exp_data = [
    ("2 năm", 2),
    ("Không yêu cầu", 0),
    ("Tới 5 năm", 0),
    ("Trên 4 năm", 4),
    ("1-3 năm", 1),
    ("", None),
    (None, None),
]
min_exp_df = spark.createDataFrame(min_exp_data, ["input", "expected"])
min_exp_result = min_exp_df.withColumn("parsed", parse_min_experience("input"))
min_exp_mismatches = min_exp_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert min_exp_mismatches.count() == 0, "parse_min_experience failed for some test cases"

# Test cases for parse_max_experience
max_exp_data = [
    ("2 năm", 2),
    ("Không yêu cầu", None),
    ("Tới 5 năm", 5),
    ("Trên 4 năm", None),
    ("1-3 năm", 3),
    ("", None),
    (None, None),
]
max_exp_df = spark.createDataFrame(max_exp_data, ["input", "expected"])
max_exp_result = max_exp_df.withColumn("parsed", parse_max_experience("input"))
max_exp_mismatches = max_exp_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert max_exp_mismatches.count() == 0, "parse_max_experience failed for some test cases"

# Test parse_employment_type
test_df = spark.createDataFrame(
    [
        ("Full-time, Part-time", ["Full-time", "Part-time"]),
        ("Full-time ,Part-time", ["Full-time", "Part-time"]),
        ("Contract ; Freelance", ["Contract", "Freelance"]),
        ("Internship", ["Internship"]),
        ("Full-time, Full-time", ["Full-time"]),
        ("", [""]),       # empty → [""]
        (None, []),       # None → []
    ],
    ["raw", "expected"]
)

result_df = test_df.select(
    "raw",
    "expected",
    F.coalesce(parse_employment_type("raw"), F.array()).alias("parsed")
)

assert result_df.filter(F.col("parsed") != F.col("expected")).count() == 0

# Test cases for parse_gender
gender_data = [
    ("Nam", "Nam"),
    ("Nữ", "Nữ"),
    ("nam", "Nam"),
    ("nữ", "Nữ"),
    ("Không yêu cầu", "Không yêu cầu"),
    ("", "Không yêu cầu"),
    (None, "Không yêu cầu"),
    ("Other", "Không yêu cầu"),
]
gender_df = spark.createDataFrame(gender_data, ["input", "expected"])
gender_result = gender_df.withColumn("parsed", parse_gender("input"))
gender_mismatches = gender_result.filter(~(F.col("parsed") == F.col("expected")))
assert gender_mismatches.count() == 0, "parse_gender failed for some test cases"

# Test cases for parse_company_size
company_size_data = [
    ("__", None),
    ("100-200", "100-200"),
    ("", ""),
    (None, None),
]
company_size_df = spark.createDataFrame(company_size_data, ["input", "expected"])
company_size_result = company_size_df.withColumn("parsed", parse_company_size("input"))
company_size_mismatches = company_size_result.filter(~(F.col("parsed").eqNullSafe(F.col("expected"))))
assert company_size_mismatches.count() == 0, "parse_company_size failed for some test cases"